In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
df = pd.read_csv("data.csv")

In [5]:
df["MK"][0]

'https://cdn.shopify.com/s/files/1/0551/4310/4592/files/0_1_b8ca2194-2c3a-4a5d-a149-11a7213fb211.png?v=1720088271'

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from PIL import Image
import os
import base64

# Initialize GPT (vision-capable)
llm = ChatOpenAI(model="gpt-4o-mini")

def ensure_supported_format(image_path: str) -> str:
    """
    Ensures the image is in JPG or PNG format.
    Converts WEBP (or other formats) to JPG automatically.
    Returns the path to the safe file.
    """
    ext = os.path.splitext(image_path)[1].lower()
    if ext in [".jpg", ".jpeg", ".png"]:
        return image_path  # already safe format
    
    # Convert unsupported formats (e.g., .webp) to .jpg
    img = Image.open(image_path).convert("RGB")
    safe_path = image_path.rsplit(".", 1)[0] + ".jpg"
    img.save(safe_path, "JPEG")
    return safe_path

def image_to_base64(image_path: str) -> str:
    """Reads a local image and returns a base64-encoded data URL."""
    with open(image_path, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode("utf-8")
    ext = os.path.splitext(image_path)[1].lower().replace(".", "")
    if ext == "jpg":
        ext = "jpeg"  # correct MIME type
    return f"data:image/{ext};base64,{b64}"

def describe_image_with_langchain(llm, image_path: str, detail_level: str = "very detailed", local_img = True):

    if local_img:    
        # Ensure we have a supported format
        safe_image_path = ensure_supported_format(image_path)
        
        # Encode local file as base64 data URL
        data_url = image_to_base64(safe_image_path)
    else:
        data_url = image_path
    
    # Build multimodal input
    message = HumanMessage(
        content=[
            {"type": "text", "text": f"Describe this image in {detail_level} for Midjourney without command, mentioning all visible objects, colors, context, and actions"},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    )
    
    # Call GPT
    response = llm.invoke([message])
    return response.content

description = describe_image_with_langchain(llm, "charmingdesign.webp")
print("Detailed description:\n", description)


Detailed description:
 The image showcases a cozy winter-themed bedding set featuring a delightful arrangement of decorative pillows and a quilt. The background is a soft, pastel blue hue that complements the entire design.

In the foreground, there are several pillows adorned with charming embroidered snowmen. Each snowman character has a unique design: one wears a bright red scarf and hat with a cheerful smile, while another sports a blue scarf and hat, complete with a buttoned coat. The snowmen are depicted with rosy cheeks and expressive eyes, exuding a playful and joyful atmosphere.

Around the snowmen, various embellishments add to the festive vibe. There are clusters of embroidered ornaments in contrasting colors like deep red and shimmering silver. Some of the accents include snowflakes and small floral designs, enhancing the overall winter theme. The edges of the quilt and pillows are finished with delicate stitching, showcasing a quilted texture that invites warmth and comfor

In [10]:
description = describe_image_with_langchain(llm, df["MK"][0], local_img = False)
print("Detailed description:\n", description)

Detailed description:
 The image features a vibrant and playful bedspread adorned with a patchwork of colorful cats. 

1. **Colors and Patterns**: The quilt showcases a variety of hues, including oranges, yellows, greens, blues, and purples, all arranged in a whimsical pattern featuring cartoonish cat faces. Each cat is uniquely designed, with distinguishing features like large, expressive eyes and quirky expressions.

2. **Positioning and Texture**: The quilt is draped over a neatly made bed, its texture suggesting a soft, quilted finish. It displays intricate stitching details, particularly in the swirls and lines that add depth to the design.

3. **Bed and Surroundings**: The bed is located in a cozy room with a headboard that has a light beige fabric appearance. A set of white pillows rests at the top of the bed, and a singular decorative pillow featuring a cat motif and bold eyes sits just behind the main quilt on the bed, adding a touch of complementing color.

4. **Lighting and 